In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets

In [2]:
#np.random.seed(5)
iris = datasets.load_iris()
X = pd.DataFrame(iris.data[50:151, 2:4])  # we only take the last two features.
y = pd.DataFrame(iris.target[50:151])


In [3]:
#Scaling
X_0min=X.iloc[:,0].min()
X_1min=X.iloc[:,1].min()
X_0max=X.iloc[:,0].max()
X_1max=X.iloc[:,1].max()

X_a = (X.iloc[:,0]-X_0min)/(X_0max-X_0min)
X_b = (X.iloc[:,1]-X_1min)/(X_1max-X_1min)

X_final=pd.DataFrame(columns=['0','1'])
X_final['0']=X_a
X_final['1']=X_b

#Learing Rate- Alpha
alpha = 0.5


In [4]:
X_final

,0,1
0,0.435897,0.266667
1,0.384615,0.333333
2,0.487179,0.333333
3,0.256410,0.200000
4,0.410256,0.333333
5,0.384615,0.200000
6,0.435897,0.400000
7,0.076923,0.000000
8,0.410256,0.200000
9,0.230769,0.266667


In [5]:
#Rescaling the class variable and setting error to zero
y=y-1
error=0

In [6]:
#Activation Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
#forward propagation
def forwardpp(a1,y,theta_1,theta_2):
    #Hidden Layer
    z2 = np.matmul(theta_1,a1.transpose())
    a2 = []
    a2.append(1.0)
    a2.append(sigmoid(z2[0]))
    a2.append(sigmoid(z2[1]))
    a2 = np.array(a2)
    
    #Output
    z3 = np.matmul(theta_2,a2.transpose())
    a3 = []
    a3.append(sigmoid(z3[0]))
    a3 = np.array(a3)
    J= (y*np.log10(a3[0]) +((1.0-y)*np.log10(1.0-a3[0])))
    return a2,a3

In [8]:
#back propagation
def backpp(a2,a3,y,theta1,theta2):
    #Output Layer
    delta_J_z1_lyr3 = ((y/a3[0]) -(1-y)/(1-a3[0]))*(a3[0]*(1-a3[0]))
    delta_J_theta1_lyr2 = delta_J_z1_lyr3*a2

    delta_J_theta2 = np.array([delta_J_theta1_lyr2])
    #theta2 = theta2 - (alpha * delta_J_theta2)
    
    #Hidden Layer
    delta_J1_z1_lyr2 = ((y/a3[0])-(1-y)/(1-a3[0]))*(a3[0]*(1-a3[0]))*theta2[0][1]*(a2[0]*(1-a2[0]))
    delta_J_z1_lyr2 = delta_J1_z1_lyr2 
    delta_J_theta1_lyr1 = delta_J_z1_lyr2*a1
    
    delta_J1_z2_lyr2 = ((y/a3[0])-(1-y)/(1-a3[0]))*(a3[0]*(1-a3[0]))*theta2[0][1]*(a2[1]*(1-a2[1]))
    delta_J_z2_lyr2 = delta_J1_z2_lyr2 
    delta_J_theta2_lyr1 = delta_J_z2_lyr2*a1
    
    delta_J_theta1 = np.array([delta_J_theta1_lyr1,delta_J_theta2_lyr1])
    
    #theta1 = theta1 - (alpha * delta_J_theta1)
    return delta_J_theta1,delta_J_theta2

In [9]:
J =[]
iterations = []
m = 99
y_pred=np.ones(100)
for i in range(X.shape[0]):
    t1,t2 = [],[]
    X_test=np.asarray(pd.DataFrame(X_final.loc[i].reshape(1,2)))
    y_test=pd.DataFrame(y.loc[i])

    X_train=pd.DataFrame(np.delete(np.asarray(X_final),i,axis=0))
    y_train=pd.DataFrame(np.delete(np.asarray(y),i,axis=0))
    #print(X_train.shape)
    # add the x_0 column bo both X_train and X_test

    n = X_train.shape[1]
    # convert from pandas DataFrame to ndarray
    X_train = X_train.as_matrix()
    #X_test = X_test.as_matrix()
    y_train = y_train.as_matrix()
    y_test = y_test.as_matrix()
    
   

    #Randomising the initial weights
    theta1 = np.random.rand(2,3)
    theta2 = np.random.rand(1,3)

    for index_iter in range(500):
        if index_iter % 10 == 0:
            print ("iteration{}".format(str(index_iter)))
        t1= np.zeros((2, 3))
        t2= np.zeros((1, 3))
        for index_sample in range(m):
            a1=[]
            a1.append(1.0)
            a1.append(X_train[index_sample][0])
            a1.append(X_train[index_sample][1])
            a1=np.asarray(a1)
            a2,a3 = forwardpp(a1,y_train[index_sample],theta1,theta2)
            #J.append(j)
            a2 = np.asarray(a2)
            iterations.append(i)
            theta1n,theta2n = backpp(a2,a3,y_train[index_sample],theta1,theta2)
            t1+=theta1n
            t2+=theta2n

        t1= t1/m
        t2= t2/m
        theta1= theta1 - (alpha*t1)
        theta2= theta2 - (alpha*t2)
    
    a1=[]
    a1.append(1.0)
    a1.append(X_test[0][0])
    a1.append(X_test[0][1])
    a1=np.asarray(a1)
    a2,a3 = forwardpp(a1,y_test[0],theta1,theta2)
    Y_target=a3
    if (Y_target >= 0.5):
        y_pred[i]=1
    else:
        y_pred[i]=0

    if(y_pred[i] != y_test):
        error=error+1


    

C:\Users\shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteratio

iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteratio

iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteratio

iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iterat

iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration4

iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration4

iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
it

iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteration270
iteration280
iteration290
iteration300
iteration310
iteration320
iteration330
iteration340
iteration350
iteration360
iteration370
iteration380
iteration390
iteration400
iteration410
iteration420
iteration430
iteration440
iteration450
iteration460
iteration470
iteration480
iteration490
iteration0
iteration10
iteration20
iteration30
iteration40
iteration50
iteration60
iteration70
iteration80
iteration90
iteration100
iteration110
iteration120
iteration130
iteration140
iteration150
iteration160
iteration170
iteration180
iteration190
iteration200
iteration210
iteration220
iteration230
iteration240
iteration250
iteration260
iteratio

In [10]:
print('Error Rate:{} %'.format(error/100))

Error Rate:0.08 %


In [11]:
y_pred

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

We observe that the results of neural networks and logistic regression are very similar. Given the results considering the random initial thetas , we can say that logistic was slightly better than neural.